In [1]:
from binance.client import Client
import numpy as np
import pandas as pd
import smtplib
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import sys
import datetime

In [2]:
# !pip install pywhatkit
import pyautogui
import pywhatkit

In [3]:
API_KEY = 'API_KEY'
API_SECRET = 'API_SECRET'

In [4]:
client = Client(API_KEY, API_SECRET)

In [5]:
SYMBOLS = ('MATICBUSD', 'AVAXBUSD', 'ENJBUSD', 'TLMBUSD', 'ONTBUSD', 'BNBBUSD', 'LINKBUSD','CHZBUSD'
           ,'BTTBUSD','NEOBUSD','ALICEBUSD','SOLBUSD','ETHBUSD','UNIBUSD','BTCBUSD')

In [6]:
def historical_to_histdf(historical):
    hist_df = pd.DataFrame(historical)
    hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
    hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
    hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
    hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)
    return hist_df

In [7]:
def rsi_cal(hist_df):
    delta = hist_df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)

    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()

    rs = ema_up/ema_down
    hist_df['RSI'] = 100 - (100/(1 + rs))
    return hist_df

In [9]:
def send_whatsup_mes(rsi_regular_historical,SYMBOL):
    yazi = "Guncel "+SYMBOL+" RSI verisidir."+np.array_str(rsi_regular_historical['RSI'].tail(1).values)
    #pyautogui.typewrite(yazi)
    #pyautogui.press("enter")
    pywhatkit.sendwhatmsg_instantly(phone_no="+00000000000",message=yazi,wait_time=2)
    time.sleep(3)
    pyautogui.click(936,989)
    time.sleep(2)
    pyautogui.press("enter")
    time.sleep(2)
    pyautogui.hotkey('ctrl','w')
    pyautogui.press("enter")

In [ ]:
while True:
    for SYMBOL in SYMBOLS:        
        historical = client.get_historical_klines(SYMBOL,Client.KLINE_INTERVAL_15MINUTE,start_str='4 September 2021')
        regular_historical = historical_to_histdf(historical)
        rsi_regular_historical = rsi_cal(regular_historical)
        send_whatsup_mes(rsi_regular_historical,SYMBOL)
    time.sleep(60 * 2)